In [3]:
import numpy as np
import pandas as pd
import cv2
import os, os.path
import matplotlib.pyplot as plt
import seaborn as sns
#from keras.datasets import cifar10
#from google.colab.patches import cv2_imshow

In [37]:
os.listdir("train/")[0:10]

['1.png',
 '10.png',
 '100.png',
 '1000.png',
 '10000.png',
 '10001.png',
 '10002.png',
 '10003.png',
 '10004.png',
 '10005.png']

In [38]:
os.path.splitext(os.listdir("train")[1])

('10', '.png')

In [63]:
train_df = pd.DataFrame(columns=range(1024))

In [62]:
grayy

array([[208, 152, 154, ..., 115, 121, 126]], dtype=uint8)

In [65]:
train_df.loc[0] = grayy

ValueError: cannot set a row with mismatched columns

In [29]:
train_df = pd.DataFrame()
for img in os.listdir("train"):
    image = cv2.imread(img)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    train_df = train_df.append(, ignore_index=True)

In [12]:
cv2.imshow('Original image',image)
cv2.imshow('Gray image', gray_image)
  
cv2.waitKey(0)
cv2.destroyAllWindows()

In [79]:
grayy = gray_image.reshape(1, -1)

In [80]:
pd.DataFrame(grayy)

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,208,152,154,158,157,158,158,158,157,158,...,103,104,105,104,103,105,107,115,121,126
